In [1]:
cd ..

/Users/johanbarreiro/Documents/GitHub/vl_optimizer


In [2]:
import pandas as pd
from pyomo.environ import NonNegativeReals, NonNegativeIntegers
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, Set, Param, NonNegativeReals, minimize

Read the parameter data:

In [3]:
# Read the CSV file into a DataFrame
df = pd.read_csv('optimizer/optimizer_input/parameters.csv')

df = df.dropna()

# Choose the 24-hour period you want to optimize (for example, starting on 2024-07-01 00:00:00)
start_date = '2023-06-01 00:00:00'
end_date = '2023-06-02 00:00:00'

# Filter the data for the chosen 24-hour period
data = df[(df['Time'] >= start_date) & (df['Time'] < end_date)]

# Ensure that the data is sorted by timestamp and reset the index
data = data.sort_values(by='Time').reset_index(drop=True)

# Check the filtered data
data


,Time,production_schedule,compressor_consumption,operational_presence,testing_schedule,active_computers,maintenance_status,lightbulbs_active,workload,volume_production_waste,fabric_in_chamber,active_wall_plugs,number_of_workers,heat_index,price_mWh,Hour,efficiency_adjusted_power,technological_centers_consumption
0,2023-06-01 00:00:00,0,1.008715,0.000000,0.091031,0,0,0,0.000000,0,0.000000,0,0,14.27,94.09,0,0.608466,1.001112
1,2023-06-01 01:00:00,31,16.339983,4.370405,15.624779,1,15,1,23.990761,17,11.078917,2,0,13.77,87.00,1,0.609133,30.909301
2,2023-06-01 02:00:00,0,16.491206,3.889101,17.954943,1,24,1,25.457470,15,3.723168,0,73,13.60,87.00,2,0.609876,28.756900
3,2023-06-01 03:00:00,0,16.465381,4.362756,11.717233,1,1,2,19.707243,11,5.162315,1,0,13.77,89.01,3,0.610317,28.556674
4,2023-06-01 04:00:00,21,17.650720,5.727740,19.267657,2,34,3,22.953785,13,9.049066,2,0,14.27,81.00,4,0.609933,28.556675
5,2023-06-01 05:00:00,12,17.272565,3.986613,6.058799,0,0,2,19.074010,9,7.968235,2,0,15.06,80.10,5,0.609537,28.381483
6,2023-06-01 06:00:00,12,27.863739,3.245102,21.827980,2,45,1,17.939803,13,5.510599,1,0,16.10,88.10,6,0.609557,29.707961
7,2023-06-01 07:00:00,41,52.271777,5.813971,14.187327,1,24,3,21.181540,18,9.834054,3,0,17.31,98.50,7,0.609516,27.355346
8,2023-06-01 08:00:00,31,55.474211,4.728541,18.233220,1,44,2,18.413512,17,7.561134,2,0,18.60,99.07,8,0.609570,28.356455
9,2023-06-01 09:00:00,32,54.667023,3.414813,15.624900,2,35,2,21.713372,24,7.653581,3,29,19.89,87.30,9,0.609632,28.857011


Create a model:

In [4]:
# Define the model
model = ConcreteModel(name="Daily_Price_Optimizer")

# Display the model
print(model)

Daily_Price_Optimizer


**Define Sets:** 

In [5]:
# Define the set of hours
model.hours = Set(initialize=range(0, 24))

**Define Parameters:**
Parameters are values that are given to us, and we don’t have control over them.

In [6]:
# Define the parameters and initialize them with data from the CSV
model.production_schedule = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['production_schedule'].to_dict())

model.maintenance_status = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['maintenance_status'].to_dict())

model.volume_production_waste = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['volume_production_waste'].to_dict())

model.number_of_workers = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['number_of_workers'].to_dict())

model.heat_index = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['heat_index'].to_dict())

model.efficiency_adjusted_power = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['efficiency_adjusted_power'].to_dict())

model.compressors_energy = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['compressor_consumption'].to_dict())

model.operational_presence = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['operational_presence'].to_dict())

model.fabric_in_chamber = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['fabric_in_chamber'].to_dict())

model.testing_schedule = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['testing_schedule'].to_dict())

model.workload = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['workload'].to_dict())

model.technological_centers_energy = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['technological_centers_consumption'].to_dict())

model.lightbulbs_active = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['lightbulbs_active'].to_dict())

model.active_wall_plugs = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['active_wall_plugs'].to_dict())

model.active_computers = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['active_computers'].to_dict())

model.price_mWh = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['price_mWh'].to_dict())

# Display the model
print(model)

Daily_Price_Optimizer


**Define Decision Variables with bounds:**
This means  x  and  y  are the variables we want to determine, and they must be non-negative (can’t be less than zero).

In [7]:


# Define the decision variables
model.power_transport_vehicles = Var(model.hours, within=NonNegativeReals, bounds=(0, 200))
model.set_point = Var(model.hours, within=NonNegativeIntegers, bounds=(18, 25))
model.num_active_chiller = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 10))
model.standby_power_down = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 1))
model.active_printers = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 3))
model.active_coffee_machines = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 3))
model.num_servers = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_network_switches_poe = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_network_switches_non_poe = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_hdds = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_ssds = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))

# Define additional decision variables for constraint outcomes
model.chiller_energy = Var(model.hours, within=NonNegativeReals)
model.data_center_energy = Var(model.hours, within=NonNegativeReals)
model.production_energy = Var(model.hours, within=NonNegativeReals)
model.uta_energy = Var(model.hours, within=NonNegativeReals)
model.office_energy = Var(model.hours, within=NonNegativeReals)

# Display the model to check the variables
print(model)

Daily_Price_Optimizer


**Define the constraints:**

In [8]:
import joblib    

chiller_model = joblib.load('equation_modeling/models/chiller_consumption_model.joblib')
data_center_model = joblib.load('equation_modeling/models/data_center_consumption_model.joblib')
office_model = joblib.load('equation_modeling/models/office_consumption_model.joblib')
production_model = joblib.load('equation_modeling/models/production_consumption_model.joblib')
uta_model = joblib.load('equation_modeling/models/uta_consumption_model.joblib')

In [9]:
chiller_intercept = chiller_model['intercept']
chiller_coefficients = chiller_model['coefficients']
chiller_features = chiller_model['features']

# Define the features as Pyomo expressions
def heat_index_set_point_diff_expr(model, h):
    return model.heat_index[h] - model.set_point[h]

# Define the chiller energy consumption constraint
def chiller_energy_constraint_rule(model, h):
    # Calculate the adjusted energy using the linear regression model
    COP_value = 3.0 
    
    adjusted_energy = ((
        chiller_intercept +
        chiller_coefficients[0] * heat_index_set_point_diff_expr(model, h) +
        chiller_coefficients[1] * model.efficiency_adjusted_power[h] +
        chiller_coefficients[2] * model.num_active_chiller[h]
    ) / (COP_value * model.efficiency_adjusted_power[h]))
    
    return model.chiller_energy[h] == adjusted_energy

# Add the constraint to the model
model.chiller_energy_constraint = Constraint(model.hours, rule=chiller_energy_constraint_rule)

In [10]:
data_center_coefficients = data_center_model['coefficients']
data_center_features = data_center_model['features']

# Define the chiller energy consumption constraint
def data_center_constraint_rule(model, h):
    
    energy = (
        data_center_coefficients['server'] * model.num_servers[h] +
        data_center_coefficients['network_switch_poe'] * model.num_network_switches_poe[h] +
        data_center_coefficients['network_switch_non_poe'] * model.num_network_switches_non_poe[h] +
        data_center_coefficients['hdd'] * model.num_hdds[h] +
        data_center_coefficients['ssd'] * model.num_ssds[h]
    ) 
    
    return model.data_center_energy[h] == energy


# Add the constraint to the model
model.data_center_energy_constraint = Constraint(model.hours, rule=data_center_constraint_rule)

In [11]:
production_intercept = production_model['intercept']
production_coefficients = production_model['coefficients']
production_features = production_model['features']

# Define the chiller energy consumption constraint
def production_energy_constraint_rule(model, h):
    
    energy = (
        production_intercept +
        production_coefficients[0] * model.power_transport_vehicles[h] +
        production_coefficients[1] * model.production_schedule[h] +
        production_coefficients[2] * model.maintenance_status[h] +
        production_coefficients[3] * model.volume_production_waste[h] +
        production_coefficients[4] * model.number_of_workers[h]
    ) 
    
    return model.production_energy[h] == energy

# Add the constraint to the model
model.production_energy_constraint = Constraint(model.hours, rule=production_energy_constraint_rule)

In [12]:
uta_intercept = uta_model['intercept']
uta_coefficients = uta_model['coefficients']
uta_features = uta_model['features']

# Define the chiller energy consumption constraint
def uta_energy_constraint_rule(model, h):
    
    energy = (
        uta_intercept +
        uta_coefficients[0] * model.operational_presence[h] +
        uta_coefficients[1] * model.fabric_in_chamber[h] +
        uta_coefficients[2] * model.testing_schedule[h] +
        uta_coefficients[3] * model.workload[h] +
        uta_coefficients[4] * model.standby_power_down[h]
    ) 
    
    return model.uta_energy[h] == energy

# Add the constraint to the model
model.uta_energy_constraint = Constraint(model.hours, rule=uta_energy_constraint_rule)

In [13]:
office_intercept = office_model['intercept']
office_coefficients = office_model['coefficients']
office_features = office_model['features']

# Define the chiller energy consumption constraint
def office_energy_constraint_rule(model, h):
    
    energy = (
        office_intercept +
        office_coefficients[0] * model.lightbulbs_active[h] +
        office_coefficients[1] * model.active_wall_plugs[h] +
        office_coefficients[2] * model.active_computers[h] +
        office_coefficients[3] * model.active_printers[h] +
        office_coefficients[4] * model.active_coffee_machines[h]
    ) 
    
    return model.office_energy[h] == energy

# Add the constraint to the model
model.office_energy_constraint = Constraint(model.hours, rule=office_energy_constraint_rule)

**Define Objective Function:**
This means we want to minimize the total cost

In [14]:


# Define the objective function
def total_cost_rule(model):
    return sum(
        (model.price_mWh[h] / 1000) * (
            model.compressors_energy[h] +
            model.technological_centers_energy[h] +
            model.chiller_energy[h] +
            model.data_center_energy[h] +
            model.production_energy[h] +
            model.uta_energy[h] +
            model.office_energy[h]
        ) for h in model.hours
    )

# Add the objective function to the model
model.total_cost = Objective(rule=total_cost_rule, sense=minimize)

**Solve the optimization problem:**

In [15]:
# Solve the model using GLPK
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Check the solver status
if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
    print("Solver found an optimal solution.")
else:
    print("Solver did not find an optimal solution. Status: ", results.solver.status)

# Display the objective function value
print("Total Cost: ", model.total_cost())

# Display the values of the decision variables
for h in model.hours:
    print(f"Hour {h}:")
    print(f"  Power Transport Vehicles: {model.power_transport_vehicles[h].value}")
    print(f"  Set Point: {model.set_point[h].value}")
    print(f"  Number of Active Chillers: {model.num_active_chiller[h].value}")
    print(f"  Standby Power Down: {model.standby_power_down[h].value}")
    print(f"  Active Printers: {model.active_printers[h].value}")
    print(f"  Active Coffee Machines: {model.active_coffee_machines[h].value}")
    print(f"  Number of Servers: {model.num_servers[h].value}")
    print(f"  Number of Network Switches (PoE): {model.num_network_switches_poe[h].value}")
    print(f"  Number of Network Switches (Non-PoE): {model.num_network_switches_non_poe[h].value}")
    print(f"  Number of HDDs: {model.num_hdds[h].value}")
    print(f"  Number of SSDs: {model.num_ssds[h].value}")
    print(f"  Chiller Energy: {model.chiller_energy[h].value}")
    print(f"  Data Center Energy: {model.data_center_energy[h].value}")
    print(f"  Production Energy: {model.production_energy[h].value}")
    print(f"  UTA Energy: {model.uta_energy[h].value}")
    print(f"  Office Energy: {model.office_energy[h].value}")
    print(f"  Compressors Energy: {model.compressors_energy[h]}")
    print(f"  Technological Centers Energy: {model.technological_centers_energy[h]}")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpo8f71v8d.glpk.raw
 --wglp /var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpnaut8igh.glpk.glp
 --cpxlp /var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmppe7pe0w6.pyomo.lp
Reading problem data from '/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmppe7pe0w6.pyomo.lp'...
120 rows, 385 columns, 384 non-zeros
240 integer variables, 24 of which are binary
1500 lines were read
Writing problem data to '/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpnaut8igh.glpk.glp'...
1470 lines were written
GLPK Integer Optimizer 5.0
120 rows, 385 columns, 384 non-zeros
240 integer variables, 24 of which are binary
Preprocessing...
11 rows, 22 columns, 22 non-zeros
22 integer variables, none of which are binary
Scaling...
 A: min|aij| =  8.255e-01  max|aij| =  1.224e+01  ratio =  1.483e+01
GM: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
EQ: 

# DO IT ALL

In [16]:
data = pd.read_csv('optimizer/optimizer_input/parameters.csv')
data['Time'] = pd.to_datetime(data['Time'])

In [17]:
from datetime import timedelta, datetime
import pandas as pd
from pyomo.environ import *

# Assuming `data` is already defined and contains the necessary data

# Create empty dataframes to store the results
hourly_results_df = pd.DataFrame()
daily_results_df = pd.DataFrame()

# Define the start and end dates for the iteration
start_date = pd.to_datetime('2023-06-01')
end_date = pd.to_datetime('2023-07-01')

# Iterate over each day
current_date = start_date

while current_date < end_date:
    # Define the 24-hour period
    start_datetime = current_date
    end_datetime = current_date + timedelta(days=1)

    # Filter the data for the current 24-hour period
    filtered_data = data[(data['Time'] >= start_datetime) & (data['Time'] < end_datetime)]
    filtered_data = filtered_data.sort_values(by='Time').reset_index(drop=True)

    # Initialize the model parameters with filtered data
    model = ConcreteModel()
    model.hours = RangeSet(0, 23)
    model.production_schedule = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['production_schedule'].to_dict())
    model.maintenance_status = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['maintenance_status'].to_dict())
    model.volume_production_waste = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['volume_production_waste'].to_dict())
    model.number_of_workers = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['number_of_workers'].to_dict())
    model.heat_index = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['heat_index'].to_dict())
    model.efficiency_adjusted_power = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['efficiency_adjusted_power'].to_dict())
    model.compressors_energy = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['compressor_consumption'].to_dict())
    model.operational_presence = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['operational_presence'].to_dict())
    model.fabric_in_chamber = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['fabric_in_chamber'].to_dict())
    model.testing_schedule = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['testing_schedule'].to_dict())
    model.workload = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['workload'].to_dict())
    model.technological_centers_energy = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['technological_centers_consumption'].to_dict())
    model.lightbulbs_active = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['lightbulbs_active'].to_dict())
    model.active_wall_plugs = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['active_wall_plugs'].to_dict())
    model.active_computers = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['active_computers'].to_dict())
    model.price_mWh = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['price_mWh'].to_dict())

    # Define the decision variables and constraints
    # Assuming the variables and constraints are already defined
    # For example:
    model.power_transport_vehicles = Var(model.hours, within=NonNegativeReals)
    model.set_point = Var(model.hours, within=NonNegativeReals)
    model.num_active_chiller = Var(model.hours, within=NonNegativeReals)
    model.standby_power_down = Var(model.hours, within=NonNegativeReals)
    model.active_printers = Var(model.hours, within=NonNegativeReals)
    model.active_coffee_machines = Var(model.hours, within=NonNegativeReals)
    model.num_servers = Var(model.hours, within=NonNegativeReals)
    model.num_network_switches_poe = Var(model.hours, within=NonNegativeReals)
    model.num_network_switches_non_poe = Var(model.hours, within=NonNegativeReals)
    model.num_hdds = Var(model.hours, within=NonNegativeReals)
    model.num_ssds = Var(model.hours, within=NonNegativeReals)
    model.chiller_energy = Var(model.hours, within=NonNegativeReals)
    model.data_center_energy = Var(model.hours, within=NonNegativeReals)
    model.production_energy = Var(model.hours, within=NonNegativeReals)
    model.uta_energy = Var(model.hours, within=NonNegativeReals)
    model.office_energy = Var(model.hours, within=NonNegativeReals)

    # Assuming the objective function is already defined
    # For example:
    def total_cost_rule(model):
        return sum(model.price_mWh[h] * (model.power_transport_vehicles[h] + model.chiller_energy[h] + model.data_center_energy[h] + model.production_energy[h] + model.uta_energy[h] + model.office_energy[h]) for h in model.hours)
    model.total_cost = Objective(rule=total_cost_rule, sense=minimize)

    # Solve the model
    solver = SolverFactory('glpk')
    results = solver.solve(model, tee=True)

    # Check the solver status
    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print(f"Solver found an optimal solution for {current_date.date()}.")
    else:
        print(f"Solver did not find an optimal solution for {current_date.date()}. Status: {results.solver.status}")

    # Store hourly results
    hourly_data = {
        'Date': [],
        'Hour': [],
        'Power Transport Vehicles': [],
        'Set Point': [],
        'Number of Active Chillers': [],
        'Standby Power Down': [],
        'Active Printers': [],
        'Active Coffee Machines': [],
        'Number of Servers': [],
        'Number of Network Switches (PoE)': [],
        'Number of Network Switches (Non-PoE)': [],
        'Number of HDDs': [],
        'Number of SSDs': [],
        'Compressor Energy': [],
        'Technological Centers Energy': [],
        'Chiller Energy': [],
        'Data Center Energy': [],
        'Production Energy': [],
        'UTA Energy': [],
        'Office Energy': [],
        'Total Cost': [],
        # # Include the parameters
        # 'Production Schedule': [],
        # 'Maintenance Status': [],
        # 'Volume Production Waste': [],
        # 'Number of Workers': [],
        # 'Heat Index': [],
        # 'Efficiency Adjusted Power': [],
        # 'Operational Presence': [],
        # 'Fabric in Chamber': [],
        # 'Testing Schedule': [],
        # 'Workload': [],
        # 'Lightbulbs Active': [],
        # 'Active Wall Plugs': [],
        # 'Active Computers': [],
        # 'Price mWh': []
    }
    for h in model.hours:
        hourly_data['Date'].append(current_date)
        hourly_data['Hour'].append(h)
        hourly_data['Power Transport Vehicles'].append(model.power_transport_vehicles[h].value)
        hourly_data['Set Point'].append(model.set_point[h].value)
        hourly_data['Number of Active Chillers'].append(model.num_active_chiller[h].value)
        hourly_data['Standby Power Down'].append(model.standby_power_down[h].value)
        hourly_data['Active Printers'].append(model.active_printers[h].value)
        hourly_data['Active Coffee Machines'].append(model.active_coffee_machines[h].value)
        hourly_data['Number of Servers'].append(model.num_servers[h].value)
        hourly_data['Number of Network Switches (PoE)'].append(model.num_network_switches_poe[h].value)
        hourly_data['Number of Network Switches (Non-PoE)'].append(model.num_network_switches_non_poe[h].value)
        hourly_data['Number of HDDs'].append(model.num_hdds[h].value)
        hourly_data['Number of SSDs'].append(model.num_ssds[h].value)
        hourly_data['Compressor Energy'].append(model.compressors_energy[h])
        hourly_data['Technological Centers Energy'].append(model.technological_centers_energy[h])
        hourly_data['Chiller Energy'].append(model.chiller_energy[h].value)
        hourly_data['Data Center Energy'].append(model.data_center_energy[h].value)
        hourly_data['Production Energy'].append(model.production_energy[h].value)
        hourly_data['UTA Energy'].append(model.uta_energy[h].value)
        hourly_data['Office Energy'].append(model.office_energy[h].value)
        hourly_data['Total Cost'].append(model.total_cost())
        # # Include the parameters
        # hourly_data['Production Schedule'].append(model.production_schedule[h])
        # hourly_data['Maintenance Status'].append(model.maintenance_status[h])
        # hourly_data['Volume Production Waste'].append(model.volume_production_waste[h])
        # hourly_data['Number of Workers'].append(model.number_of_workers[h])
        # hourly_data['Heat Index'] .append(model.heat_index[h])
        # hourly_data['Efficiency Adjusted Power'].append(model.efficiency_adjusted_power[h])
        # hourly_data['Operational Presence'].append(model.operational_presence[h])
        # hourly_data['Fabric in Chamber'].append(model.fabric_in_chamber[h])
        # hourly_data['Testing Schedule'].append(model.testing_schedule[h])
        # hourly_data['Workload'].append(model.workload[h])
        # hourly_data['Lightbulbs Active'].append(model.lightbulbs_active[h])
        # hourly_data['Active Wall Plugs'].append(model.active_wall_plugs[h])
        # hourly_data['Active Computers'].append(model.active_computers[h])
        # hourly_data['Price mWh'].append(model.price_mWh[h])

    hourly_results_df = pd.concat([hourly_results_df, pd.DataFrame(hourly_data)], ignore_index=True)

    # Store daily results  
    new_row = pd.DataFrame({
        'Date': [current_date],
        'Total Cost': [model.total_cost()]
    })
    daily_results_df = pd.concat([daily_results_df, new_row], ignore_index=True)

    # Move to the next day
    current_date += timedelta(days=1)


# Reset index of the dataframes
hourly_results_df.reset_index(drop=True, inplace=True)
daily_results_df.reset_index(drop=True, inplace=True)

# Get the current datetime in the format YYYYMMDD_HHMMSS
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")


daily_results_df['Optimized_Total_Cost'] = daily_results_df['Total Cost']
daily_results_df = daily_results_df.drop(columns=['Total Cost'])

cols_to_rename = [col for col in hourly_results_df.columns if col not in ['Date', 'Hour']]
hourly_results_df.rename(columns={col: f"optimized_{col}" for col in cols_to_rename}, inplace=True)

# Save the files to the new directory with the current datetime
hourly_results_df.to_csv(f'optimizer/optimizer_output/{current_datetime}_optimized_hourly_output.csv', index=False)
daily_results_df.to_csv(f'optimizer/optimizer_output/{current_datetime}_optimized_daily_output.csv', index=False)

print(f'{current_datetime}_optimized_hourly_output.csv output file saved.')
print(f'{current_datetime}_optimized_daily_output.csv output file saved.')

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpxl2kw67z.glpk.raw
 --wglp /var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpbf_bdr71.glpk.glp
 --cpxlp /var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpxf14xtex.pyomo.lp
Reading problem data from '/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpxf14xtex.pyomo.lp'...
1 row, 145 columns, 1 non-zero
302 lines were read
Writing problem data to '/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpbf_bdr71.glpk.glp'...
296 lines were written
GLPK Simplex Optimizer 5.0
1 row, 145 columns, 1 non-zero
Preprocessing...
~     0: obj =   0.000000000e+00  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.1 Mb (64484 bytes)
Writing basic solution to '/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/tmpxl2kw67z.glpk.raw'...
155 lines were written
Solver found an optimal solution for 2023-06-01.
GLPSOL--GLPK LP

In [18]:
import pandas as pd

# Load the data
parameters_df = pd.read_csv('optimizer/optimizer_input/parameters.csv')

# Convert 'Date' column to datetime
hourly_results_df['Date'] = pd.to_datetime(hourly_results_df['Date'])
# Combine 'Date' and 'Hour' to create 'Time' column
hourly_results_df['Time'] = hourly_results_df['Date'] + pd.to_timedelta(hourly_results_df['Hour'], unit='h')
# Ensure 'Time' is in datetime format
hourly_results_df['Time'] = pd.to_datetime(hourly_results_df['Time'])
parameters_df['Time'] = pd.to_datetime(parameters_df['Time'])  # Ensure 'Time' column in parameters_df is in datetime format

# Merge the dataframes
hourly_results_df = pd.merge(hourly_results_df, parameters_df, on='Time', how='left')

# Load ground truth data
ground_truth_hourly = pd.read_csv('optimizer/ground_truth/ground_truth_hourly.csv')

# Prefix 'actual_' to all columns in ground truth dataframe
ground_truth_hourly = ground_truth_hourly.add_prefix('actual_')

# Adjust 'Time' column after prefixing
ground_truth_hourly = ground_truth_hourly.rename(columns={'actual_Time': 'Time'})

# Original column names and new column names mapping
column_mapping = {
    "optimized_Total Cost": "optimized_Total Cost (Day)",
    "actual_energy_cost": "actual_energy_cost (Hour)",
    "actual_02 Production_Electric_Active Energy (kWh)": "actual_Production Energy",
    "optimized_Production Energy": "optimized_Production Energy",
    "actual_03 Chiller Group_Electric_Active Energy (kWh)": "actual_Chiller Energy",
    "optimized_Chiller Energy": "optimized_Chiller Energy",
    "actual_04 UTA_Electric_Active Energy (kWh)": "actual_UTA Energy",
    "optimized_UTA Energy": "optimized_UTA Energy",
    "actual_05 Compressors_Electric_Active Energy (kWh)": "actual_Compressor Energy",
    "optimized_Compressor Energy": "optimized_Compressor Energy",
    "actual_06 Offices_Electric_Active Energy (kWh)": "actual_Office Energy",
    "optimized_Office Energy": "optimized_Office Energy",
    "actual_07 Data Center_Electric_Active Energy (kWh)": "actual_Data Center Energy",
    "optimized_Data Center Energy": "optimized_Data Center Energy",
    "actual_08 Technological Centers_Electric_Active Energy (kWh)": "actual_Technological Centers Energy",
    "optimized_Technological Centers Energy": "optimized_Technological Centers Energy"
}

# Rename columns
ground_truth_hourly = ground_truth_hourly.rename(columns=column_mapping)

# Ensure 'Time' column in ground_truth_hourly is in datetime format
ground_truth_hourly['Time'] = pd.to_datetime(ground_truth_hourly['Time'])

# Merge the dataframes
evaluate_hourly = pd.merge(hourly_results_df, ground_truth_hourly, on='Time', how='left')

# Define the column order
column_order = [
    "Time",
    "optimized_Total Cost",
    "actual_energy_cost (Hour)",
    "actual_Production Energy",
    "optimized_Production Energy",
    "actual_Chiller Energy",
    "optimized_Chiller Energy",
    "actual_UTA Energy",
    "optimized_UTA Energy",
    "actual_Compressor Energy",
    "optimized_Compressor Energy",
    "actual_Office Energy",
    "optimized_Office Energy",
    "actual_Data Center Energy",
    "optimized_Data Center Energy",
    "actual_Technological Centers Energy",
    "optimized_Technological Centers Energy",
    "actual_power_transport_vehicles",
    "optimized_Power Transport Vehicles",
    "actual_set_point",
    "optimized_Set Point",
    "actual_num_active_chiller",
    "optimized_Number of Active Chillers",
    "actual_standby_power_down",
    "optimized_Standby Power Down",
    "actual_active_printers",
    "optimized_Active Printers",
    "actual_active_coffee_machines",
    "optimized_Active Coffee Machines",
    "actual_server",
    "optimized_Number of Servers",
    "actual_network_switch_poe",
    "optimized_Number of Network Switches (PoE)",
    "actual_network_switch_non_poe",
    "optimized_Number of Network Switches (Non-PoE)",
    "actual_hdd",
    "optimized_Number of HDDs",
    "actual_ssd",
    "optimized_Number of SSDs",
    "actual_production_schedule",
    "actual_maintenance_status",
    "actual_volume_production_waste",
    "actual_number_of_workers",
    "actual_heat_index",
    "efficiency_adjusted_power",
    "actual_operational_presence",
    "actual_fabric_in_chamber",
    "actual_testing_schedule",
    "actual_workload",
    "actual_lightbulbs_active",
    "actual_active_wall_plugs",
    "actual_active_computers",
    "actual_price_mWh"
]

# Reorder the columns
evaluate_hourly_reorder = evaluate_hourly[column_order]

# Save the final dataframe to a CSV file
current_datetime = pd.Timestamp.now().strftime('%Y-%m-%d_%H-%M-%S')
evaluate_hourly_reorder.to_csv(f'optimizer/evaluation/{current_datetime}_evaluate_hourly_output.csv', index=False)

print('hourly evaluation file saved')

hourly evaluation file saved


In [19]:
ground_truth_daily = pd.read_csv('optimizer/ground_truth/ground_truth_daily.csv')

ground_truth_daily = ground_truth_daily.add_prefix('actual_')

ground_truth_daily = ground_truth_daily.rename(columns={'actual_Time': 'Time'})

ground_truth_daily['Date'] = ground_truth_daily['Time']
ground_truth_daily = ground_truth_daily.drop(columns=['Time'])

evaluate_daily = pd.merge(daily_results_df, ground_truth_daily, left_on='Date', right_on='Date', how='left')

evaluate_daily.to_csv(f'optimizer/evaluation/{current_datetime}_evaluate_daily_output.csv', index=False)

print('hourly evaluation file saved')

ValueError: You are trying to merge on datetime64[ns] and object columns for key 'Date'. If you wish to proceed you should use pd.concat